# Phase 6: Proactive Pattern Detection
> _(Part of the Data Science lifecycle for uncovering high-risk clusters & anomalies)_

#### **Objective:**
Use **unsupervised learning** techniques to surface _hidden structure_, such as:
* **Alias clusters** (e.g. same entity across alt names)
* **Outlier behavior** (e.g. odd country associations)
* **Potential identity masking** strategies